In [85]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [86]:
from src.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from src.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector
from src.bias.FreqVsRefBiasDetector import FreqVsRefBiasDetector

## Importing Data and Trained Classifier

**UX**: 
- the user uploads the dataset with the features (X)
- the user uploads 
    - the sklearn model as pkl file **or**
    - a file with the predictions already created (Y). 
- the user uploads scaler and ohe (if needed)

If a model is provided, the application checks if the the provided datasets and model match in terms of column names. 

In [87]:
input_data_path = "../data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = '../data/mlflow_artifacts/'

In [88]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

In [89]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [90]:
with open("trained_model_for_testing/RF_12_200.pkl", "rb") as file:
    classifier = load(file)

/home/dev/repos/venv_gen_purpose/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dev/repos/venv_gen_purpose/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [91]:
predicted_prob = classifier.predict_proba(X_test_ohe)
predicted_values = classifier.predict(X_test_ohe)

#### Definition of conditioning variables
**UX**:
- the user selects the continuous features to be categorized and used for the creation of control groups. 

The application propose splitting cuts that make the resulting discrete distribution uniform (but also other binning procedures are possible). 

In [92]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

**UX**:
- the user selects from a drop down menu the variables to be used for the groups creation
- the user can select "check all the variables" and the application will use all the available discrete features

In [93]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [94]:
df_with_predictions = X_test.reset_index(drop=True).assign(predictions=predicted_values, predicted_probs=predicted_prob[:,1])

## Bias Detection

**UX**:
- the user selects from a drop down menu the binary variable to be used for the distance calculation
- the user selects a threshold for the distance
- the user selects a Distance (if more than one are provided)

In [95]:
bd = FreqVsFreqBiasDetector(distance="TVD", target_variable_type='probability')

**UX**:

The user has can select between two options:
- Option 1: distance between frequencies
- Option 2: distance from reference distribution
    - in this case, the user has to insert values for a reference distribution

## Option 1: Distance between frequencies

### Comparison of the two groups observed freqs

In [96]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predicted_probs',
    root_variable='x2_sex',
    threshold=None,
    n_bins=10
)

(0.05588710957653692, False, 0.016368585412256314, None)

In [97]:
#distance = max( abs(female_0_freq - male_0_freq), abs(female_1_freq - male_1_freq) )

In [98]:
#print(df_with_predictions.groupby("x2_sex").predictions.value_counts(normalize=True))
print(np.histogram(df_with_predictions.query("x2_sex==1").predicted_probs, bins=10, range=[0,1])[0]/df_with_predictions.query("x2_sex==1").shape[0])
print(np.histogram(df_with_predictions.query("x2_sex==2").predicted_probs, bins=10, range=[0,1])[0]/df_with_predictions.query("x2_sex==2").shape[0])

[0.20439189 0.42454955 0.1356982  0.06503378 0.04391892 0.04560811
 0.04054054 0.02984234 0.00844595 0.00197072]
[0.260279   0.41831865 0.12756975 0.05837004 0.03432452 0.02514684
 0.03671072 0.03120411 0.00715859 0.00091777]


### Comparison of the two groups observed freqs conditioning to other features

In [99]:
results = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predicted_probs',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables,
    threshold=0.1,
    min_obs_per_group=30,
    n_bins=10)

In [100]:
#results

**UX**:
- the application shows the results in two views:
    - overall results: all the computed distances
    - violations: only the results above the threshold
- for both, the user can order the results by number of obs, distance or group name

In [101]:
# selecting only combinations for which the distance is greater than the threshold 
violations = {k: v for k, v in results.items() if not v[2]}

- '2: Gender (1 = male; 2 = female).',
- '3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).',
- '4: Marital status (1 = married; 2 = single; 3 = others).'

In [102]:
# sorting the violations by number of observations belonging to that group
dict(sorted(violations.items(), key=lambda item: item[1], reverse=True))

{'x4_marriage==3': (95, 0.10410557184750735, False, 0.1, None),
 'x3_education==2 & x4_marriage==3': (56,
  0.17251461988304095,
  False,
  0.1,
  None),
 'x3_education==5 & x4_marriage==1': (45,
  0.1774193548387097,
  False,
  0.1,
  None),
 'x3_education==4': (40, 0.3125, False, 0.1, None)}

#### Focus analyses on particular cases
Here we want to understand/qualify some specific violation cases.

**UX**: 
- the user selects one of the available violations
- the application returns relevant details to help understand what's going on
- the user can download the results as csv file. 

In [103]:
focus_df = df_with_predictions.query("x3_education==1 & x4_marriage==1 & age_buckets==1")
bd.get_frequencies_list_from_probs(focus_df, 'predicted_probs', 
                        'x2_sex', df_with_predictions.x2_sex.unique(), 10)
#TODO: TRANSFORM IN DATAFRAME

([array([0.26829268, 0.56097561, 0.04878049, 0.04878049, 0.02439024,
         0.02439024, 0.02439024, 0.        , 0.        , 0.        ]),
  array([0.1875, 0.25  , 0.25  , 0.1875, 0.0625, 0.    , 0.0625, 0.    ,
         0.    , 0.    ])],
 [array([11, 23,  2,  2,  1,  1,  1,  0,  0,  0]),
  array([3, 4, 4, 3, 1, 0, 1, 0, 0, 0])])

In [104]:
#focus_df.groupby('x2_sex').predictions.value_counts(normalize=True)

In [105]:
#TO DELETE
from scipy.stats import entropy
from scipy.special import rel_entr
p,q = bd.get_frequencies_list_from_probs(focus_df, 'predicted_probs', 
                        'x2_sex', df_with_predictions.x2_sex.unique(), 10)[0]
p,q,entropy(p,q), rel_entr(p,q), rel_entr(p,q).sum() #entropy è sum di rel_entr. Devi modificare freq_vs_ref in modo che se c'è un inf --> diventa 0. In jensenshannon non succede

#rel_entr(p,q)[5] == np.inf
a=np.array([ 0.09612917,  0.45338975, -0.07971368, -0.06568041, -0.02295081,
                np.inf, -0.02295081,  0.        ,  0.        ,  0.        ])
a[a==np.inf]=0
a

array([ 0.09612917,  0.45338975, -0.07971368, -0.06568041, -0.02295081,
        0.        , -0.02295081,  0.        ,  0.        ,  0.        ])

## Option 2: distance from reference distribution

In [114]:
bd = FreqVsRefBiasDetector(normalization='D1', adjust_div='zero', target_variable_type='probability')

In [115]:
female_distr = [0.2,0.15,0.15,0.1,0.1,0.1,0.05,0.05,0.05,0.05] #[0.75,0.25]
male_distr = female_distr

ref_distribution = [np.array(female_distr), np.array(male_distr)]

### Comparison of the two groups w.r.t. the reference distribution

In [116]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predicted_probs',
    root_variable='x2_sex',
    threshold=0.1,
    reference_distribution=ref_distribution,
    n_bins=10
)

([0.3325369126643257, 0.3950668798613991], [False, False], 0.1)

### Comparison of the two groups w.r.t. the reference distribution conditioning to other features

In [117]:
results = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions,
    target_variable='predicted_probs',
    root_variable='x2_sex',
    conditioning_variables=conditioning_variables,
    threshold=0.1,
    min_obs_per_group=30,
    reference_distribution=ref_distribution,
    n_bins=10)

In [118]:
results

{'x3_education==1': (3119,
  [0.3733894163213447, 0.33464884313930365],
  [False, False],
  0.1),
 'x3_education==3': (1499,
  [0.33639915729869396, 0.3692037899778381],
  [False, False],
  0.1),
 'x3_education==2': (4250,
  [0.3198385486496075, 0.36654921110225525],
  [False, False],
  0.1),
 'x3_education==4': (40,
  [-0.8458045172856505, -0.7337623216957061],
  [True, True],
  0.1),
 'x3_education==5': (75,
  [-0.1938359615497356, -0.13731407304501708],
  [True, True],
  0.1),
 'x3_education==6': (14, None, 'Not enough observations'),
 'x3_education==0': (3, None, 'Not enough observations'),
 'x4_marriage==1': (4065,
  [0.32083718972939557, 0.3693872204325538],
  [False, False],
  0.1),
 'x4_marriage==2': (4822,
  [0.3440863424267153, 0.2835634401991457],
  [False, False],
  0.1),
 'x4_marriage==3': (95,
  [0.40933006255278737, 0.31079226197268917],
  [False, False],
  0.1),
 'x4_marriage==0': (18, None, 'Not enough observations'),
 'x3_education==1 & x4_marriage==1': (1106,
  [0.38

In [119]:
# selecting only combinations for which the distance is greater than the threshold 
violations = {k: v for k, v in results.items() if (not v[2][0] or not v[2][1])}
len(violations)

12